# <span style='font-family:Garamond'> <b>Voice as Data: Python Process</b>
<span style='font-family:Garamond'>
By: S. Zhang and M. Gomies
<br>
&#9701; Beta Ver.: 0.1
<br>
&#9701; This is only a pre-eliminary work.
<br>
&#9701; This page will be periodically updated and may provide different process/steps.

In [1]:
### Library ###
import sys
import os
import pandas as pd
import numpy as np
import statistics
import parselmouth
from parselmouth.praat import call
from scipy.stats.mstats import zscore
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings("ignore")

from datetime import date
print("Updated by:",date.today().strftime("%m/%d/%y"), "(M/D/Y)")

Updated by: 03/20/24 (M/D/Y)


## <span style='font-family:Garamond'> <b>Defining Functions</b>
<span style='font-family:Garamond'>
In this section, we will define several functions primarily focused on extracting pitch to obtain essential voice parameters.

In [2]:
##########################################
####### Function: Progress Counter #######
##########################################
def update_progress(job_title, progress):
    length = 20 # modify this to change the length
    block = int(round(length*progress))
    msg = "\r{0}: [{1}] {2}%".format(job_title, "*"*block + "-"*(length-block), round(progress*100, 2))
    if progress >= 1: msg += " DONE\r\n"
    sys.stdout.write(msg)
    sys.stdout.flush()

In [3]:
##############################################
####### Function: Voice Data Extractor #######
##############################################

### 1. This function measures duration, pitch, HNR, jitter, and shimmer
def measurePitch(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID) # Read the sound
    duration = call(sound, "Get total duration") # Read the duration
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #Create a praat pitch object
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    meanF0 = call(pitch, "Get mean", 0, 0, unit) # Get mean pitch
    stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit) # Get the standard deviation
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1)
    hnr = call(harmonicity, "Get mean", 0, 0)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    return duration, meanF0, stdevF0, hnr, rapJitter, apq3Shimmer



### 2. This function measures formants at each glottal pulse
def measureFormants(sound, wave_file, f0min,f0max):
    sound = parselmouth.Sound(sound) # Read the sound
    pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    
    formants = call(sound, "To Formant (burg)", 0.0025, 5, 5500, 0.025, 50)
    numPoints = call(pointProcess, "Get number of points")

    f1_list = []
    f2_list = []
    f3_list = []
    f4_list = []
    
    # Measure formants only at glottal pulses
    for point in range(0, numPoints):
        point += 1
        t = call(pointProcess, "Get time from index", point)
        f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
        f4 = call(formants, "Get value at time", 4, t, 'Hertz', 'Linear')
        f1_list.append(f1)
        f2_list.append(f2)
        f3_list.append(f3)
        f4_list.append(f4)
    
    f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
    f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']
    f3_list = [f3 for f3 in f3_list if str(f3) != 'nan']
    f4_list = [f4 for f4 in f4_list if str(f4) != 'nan']
    
    # Calculate mean formants across pulses
    f1_mean = statistics.mean(f1_list)
    f2_mean = statistics.mean(f2_list)
    f3_mean = statistics.mean(f3_list)
    f4_mean = statistics.mean(f4_list)
    
    # Calculate median formants across pulses
    f1_median = statistics.median(f1_list)
    f2_median = statistics.median(f2_list)
    f3_median = statistics.median(f3_list)
    f4_median = statistics.median(f4_list)
    
    return f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median



### 3. This function runs a 2-factor Principle Components Analysis (PCA) on Jitter and Shimmer (Additional)
def runPCA(df):
    # z-score the Jitter and Shimmer measurements
    measures = ['rapJitter', 'apq3Shimmer']
    x = df.loc[:, measures].values
    x = StandardScaler().fit_transform(x)
    # PCA
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents, columns = ['JitterPCA', 'ShimmerPCA'])
    principalDf
    return principalDf
    

## <span style='font-family:Garamond'> <b>Sample Process</b>
<span style='font-family:Garamond'>
In this section, our objective is to construct a dataframe using sample voice data extracted from a real-life trial dataset.

In [4]:
#############################
##### RLT Dataset - Lie #####
#############################
data_dir = "./Data/rlt_lie/"

### Removing .DS_Store temporary file (Macbook OS)
try:
    os.remove(data_dir+".DS_Store") #
except:
    pass

list_1= os.listdir(data_dir)
list_1.sort()

### Creating list of file names
file_list = []
for item in list_1:
    file_id=item
    file_list.append(file_id)

### Creating initial panda frame
labels=pd.DataFrame(file_list)
labels=labels.rename(columns={0: "voice_id"})

### Creating lists to store parameter values ###
file_list = []
duration_list=[]
mean_F0_list = []
sd_F0_list = []
hnr_list = []
voice_id=[]
rapJitter_list=[]
apq3Shimmer_list=[]
f1_mean_list = []
f2_mean_list = []
f3_mean_list = []
f4_mean_list = []
f1_median_list = []
f2_median_list = []
f3_median_list = []
f4_median_list = []

counter = 1


### Loop process ###
for pitch_item in list_1:
    wave_file=data_dir+str(pitch_item)
    voice_id=pitch_item
    sound = parselmouth.Sound(wave_file)
    (duration, meanF0, stdevF0, hnr, rapJitter, apq3Shimmer) = measurePitch(sound, 75, 500, "Hertz")
    (f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median) = measureFormants(
    sound, wave_file, 75, 500)
    file_list.append(voice_id) # Make an ID list
    duration_list.append(duration)
    mean_F0_list.append(meanF0) # Make a mean F0 list
    sd_F0_list.append(stdevF0) # Make a sdF0 list
    hnr_list.append(hnr) # Make an hnr list
    rapJitter_list.append(rapJitter) # Make a jitter list
    apq3Shimmer_list.append(apq3Shimmer) # Make a shimmer list
    ### Add the formant data (mean and median)
    f1_mean_list.append(f1_mean)
    f2_mean_list.append(f2_mean)
    f3_mean_list.append(f3_mean)
    f4_mean_list.append(f4_mean)
    f1_median_list.append(f1_median)
    f2_median_list.append(f2_median)
    f3_median_list.append(f3_median)
    f4_median_list.append(f4_median)
    
    update_progress(data_dir, counter/len(list_1))
    counter+=1

./Data/rlt_lie/: [********************] 100.0% DONE


In [7]:
### Creating dataframe for the obtained features ###
df = pd.DataFrame(np.column_stack([file_list, duration_list, mean_F0_list, sd_F0_list, hnr_list, rapJitter_list,apq3Shimmer_list, f1_mean_list, f2_mean_list, f3_mean_list, f4_mean_list, f1_median_list, f2_median_list, f3_median_list, f4_median_list ]), 
                               columns=['voice_id', 'duration', 'meanf0hz', 'stdevf0hz', 'hnr', 'rapJitter', 'apq3Shimmer', 'f1mean', 'f2mean', 'f3mean', 'f4mean', 'f1median', 'f2median', 'f3median', 'f4median'])  #Add these lists in the right order

### Display results
display(df)

,voice_id,duration,meanf0hz,stdevf0hz,hnr,rapJitter,apq3Shimmer,f1mean,f2mean,f3mean,f4mean,f1median,f2median,f3median,f4median
0,trial_lie_002_2.wav,62.05891156462585,184.19869247757455,39.14006156816869,12.289649384641574,0.009342248021001044,0.04409368620558818,504.9814705519435,1930.9600308264644,3078.6471891918673,4220.209096224229,470.35397400748707,2052.6846996801655,3168.5434527245134,4310.178790161315
1,trial_lie_004_2.wav,11.56267573696145,185.8491914473164,52.24085177822427,13.271185547876659,0.008949568470242069,0.039319146173288085,564.0875356782922,2001.1550212122017,3053.0373152519373,4269.403351880047,548.7846886819145,2077.799848461044,3143.0164262327944,4374.932774795618
2,trial_lie_005_2.wav,53.39732426303855,180.6241269622623,44.27494501605771,13.742172620880268,0.008717087449825207,0.041163342848088855,510.0802181034382,1979.6537354895938,3109.326909365539,4258.927400571925,451.76773236512145,2082.286179059923,3178.7093334440856,4315.147429556398
3,trial_lie_006_2.wav,13.539455782312926,246.23929343973282,52.37928664554057,13.561179551897173,0.009141820780667933,0.0463790867618145,508.5137062002759,1760.2298648807002,2876.5098514514207,4034.580498284746,486.9766800057406,1753.4969045127666,2980.793257779519,4087.795033713896
4,trial_lie_007_2.wav,39.16328798185941,182.3883505147054,34.95254102132246,10.184409678139003,0.010113507172762801,0.07894417010999803,555.3715499964987,1889.4572328901738,2887.544572603458,4008.1015789862618,477.4642832578372,1831.2144895189483,2879.388733988917,3988.156575359187
5,trial_lie_008_2.wav,6.500362811791383,165.65770378997738,40.08555451918304,6.52154971271631,0.019937385775475085,0.08967133293662725,488.57095051209535,1915.4919913603887,2878.9960062430127,4019.065064124469,394.0947569112399,1875.3704503027368,2860.628035884073,4019.4481912677916
6,trial_lie_009_2.wav,15.75374149659864,169.83136437115323,49.484952448591685,8.501594608919845,0.013226447304324495,0.07006585461189688,566.7457936264967,1837.5650105293066,2857.212965256784,4044.110636224801,455.05430443283973,1753.4945806369792,2862.771901376532,4026.051875400706
7,trial_lie_010_2.wav,21.976031746031747,165.9950328409444,46.482793206829705,8.477886279992486,0.012755569920517917,0.08063971688329033,548.2521764363747,1810.2295292170384,2812.2199988398274,4017.3196990936462,482.0317584472687,1767.2879616318885,2817.9780067229012,3977.891139921225
8,trial_lie_011_2.wav,33.64267573696145,200.81378492660767,40.65185916566651,8.306759977355838,0.015601093172737647,0.0801884905829367,539.7382162046015,1940.8795686596227,2906.398027921339,4024.05284156446,472.5492498829987,1903.9170309336448,2889.360945739658,3977.3961383297906
9,trial_lie_012_2.wav,7.077868480725623,178.65647665692566,44.95453100723351,9.451172904604933,0.017975195436645306,0.09052209263993037,512.9640877472941,1992.7701280292633,2955.1149445004794,4057.124180513125,333.01198319326056,1910.7373536893558,2921.644001871342,4008.082508729499
